In [2]:
import os
from newspaper import Article
from bs4 import BeautifulSoup
from six.moves.urllib.parse import urlparse
import urllib
import sys
import time
import nltk
import glob
import pandas as pd
import requests
import spacy
import random
# from googlesearch import search
from langdetect import detect
import re
import pickle
import math
import numpy as np
import collections
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
from tabulate import tabulate
from IPython.display import display, HTML

In [13]:
path = r'C:\Users\aaberkan\OneDrive - UGent\Scripts\GDPR-Compliance in Web Applications\data\Crunchbase\Cloud'
filenames = glob.glob(path + "/*.csv")
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))
    
crunch_data = pd.concat(dfs, ignore_index=True)

In [14]:
crunch_data

,Organization Name,Organization Name URL,Full Description,Industries,Website,Headquarters Location,Description,CB Rank (Company),SEMrush - Monthly Visits,SEMrush - Average Visits (6 months),...,Founded Date,Founded Date Precision,Number of Founders,Number of Employees,Founders,Apptopia - Number of Apps,Apptopia - Downloads Last 30 Days,Aberdeen - IT Spend,Aberdeen - IT Spend Currency,Aberdeen - IT Spend Currency (in USD)
0,Ex Libris,https://www.crunchbase.com/organization/ex-libris,Ex Libris Group is a leading provider of libra...,"Apps, Cloud Computing, Enterprise Software, In...",http://www.exlibrisgroup.com,"Ballerup, Hovedstaden, Denmark",Ex Libris Group is a leading provider of libra...,"165,720","13,961,244","14,073,988.5",...,1986-01-01,year,2.0,501-1000,"Azriel Morag, Evgeniy Larionov",19.0,"3,443",NaN,NaN,NaN
1,Exact,https://www.crunchbase.com/organization/exact,Exact is a global supplier of cloud business s...,"Accounting, Cloud Computing, CRM, Enterprise R...",http://www.exact.com,"Delft, Zuid-Holland, The Netherlands",Exact is a company that provides cloud based b...,"229,780","622,387","672,910.5",...,1984-07-23,day,3.0,1001-5000,"Arco Van Nieuwland, Eduard Hagens, Irfan Verdia",32.0,"11,294",NaN,NaN,NaN
2,Xperience,https://www.crunchbase.com/organization/xperie...,Xperience provides software solutions within E...,"Cloud Computing, Consulting, CRM, Enterprise R...",https://www.xperience-group.com/,"Antrim, Antrim, United Kingdom",Xperience provides software solutions within E...,"187,002","402,949","198,967.83",...,1969-01-01,year,NaN,101-250,NaN,NaN,NaN,6256711.0,USD,6256711.0
3,Sage,https://www.crunchbase.com/organization/sage-c1e4,Sage DPW-Software is now implementing over 100...,"Cloud Computing, Computer, Private Cloud, Secu...",https://www.sagedpw.at/,"Vienna, Wien, Austria",Sage provides software for HR services.,"362,734","319,471","219,712.33",...,1972-01-01,year,NaN,101-250,NaN,NaN,NaN,NaN,NaN,NaN
4,Novahé,https://www.crunchbase.com/organization/novahé,NaN,"Cloud Computing, Information Technology, IT In...",https://www.novahe.fr/,"Orléans, Centre, France",Novahé is an information technology company sp...,"930,723","288,214",NaN,...,1986-06-30,day,NaN,51-100,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21235,BackupAddict,https://www.crunchbase.com/organization/backup...,NaN,"Cloud Storage, Information Technology, Profess...",https://www.backupaddict.com/,"Jersey Shore, Pennsylvania, United States",BackupAddict renders online backup plans to pr...,"2,145,360",NaN,NaN,...,NaN,NaN,NaN,11-50,NaN,NaN,NaN,NaN,NaN,NaN
21236,"NetConvergence, Inc.",https://www.crunchbase.com/organization/netcon...,NaN,"Cloud Storage, Enterprise Software, Informatio...",http://www.netconvergence.com,"Santa Clara, California, United States","NetConvergence, Inc. is an intellectual proper...","2,215,033",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21237,MyLabBook.com,https://www.crunchbase.com/organization/mylabb...,NaN,"Cloud Data Services, Cloud Storage, Software",https://www.mylabbook.com,"Texas, South Carolina, United States",MyLabBook.com is a company that facilitates op...,"2,222,155",NaN,NaN,...,1994-01-01,year,NaN,1-10,NaN,NaN,NaN,NaN,NaN,NaN
21238,Radmedix,https://www.crunchbase.com/organization/radmedix,NaN,"Cloud Storage, Manufacturing, Medical Device, ...",https://radmedix.com,"Dayton, Ohio, United States",Radmedix is an x-ray manufacturer that provide...,NaN,NaN,NaN,...,NaN,NaN,NaN,11-50,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
# crunch_data[0:7243]
crunch_classified.loc[crunch_data['Organization Name'] == "Fuze, an 8x8 company"]

,Organization Name,Organization Name URL,Full Description,Industries,Website,Headquarters Location,Description,CB Rank (Company),SEMrush - Monthly Visits,SEMrush - Average Visits (6 months),...,Aberdeen - IT Spend,Aberdeen - IT Spend Currency,Aberdeen - IT Spend Currency (in USD),PP URL,DPO,Purpose,Acquired data,Data sharing,Rights,PP text
1077,XTRM,https://www.crunchbase.com/organization/xtrm,Global Payment Platform XTRM is a powerful glo...,"Cloud Computing, Finance, Financial Services, ...",http://www.xtrm.com,"Miami, Florida, United States",XTRM is a secure trusted global payment and FX...,"167,636","80,299","60,393",...,NaN,NaN,NaN,https://www.xtrm.com/web/privacy.aspx,0,1,1,1,0,"['privaci polici', 'your privaci right', 'thi ..."
1077,Swift Systems,https://www.crunchbase.com/organization/swift-...,Swift Systems is a Maryland-based IT consultin...,"Cloud Computing, Consulting",http://www.swiftsystems.com,"Frederick, Maryland, United States",Swift Systems is a regional businesses as a pr...,"484,803","5,475","3,919.83",...,158479.0,USD,158479.0,https://www.swiftsystems.com/privacy-policy/,1,1,0,1,0,"['privaci polici', 'last updat august swift sy..."
1077,Zepcam,https://www.crunchbase.com/organization/zepcam,NaN,"Cloud Storage, Manufacturing",https://zepcam.com/,"Amsterdam, Noord-Holland, The Netherlands",Zepcam is a manufacturing firm that manufactur...,"298,013","8,447","4,867.33",...,NaN,NaN,NaN,https://zepcam.com/privacy-cookies/,1,1,1,0,1,"['welcom to wwwzepcamcom', 'By use thi websit ..."


In [ ]:
# PART 1 (0:8339), corrected (0: 8339 "Omnistruct Inc" is the last)
# scraped final: 1746 (crunch_classified_p1.csv)
crunch_classified_p1 = pd.read_csv("crunch_classified_p1.csv", sep='\t', encoding='utf-8')

# PART 2 (4244:16661) ("Cloud&Heat Technologies" is the last)
# scraped final: 2476 (crunch_classified_p2.csv)

# Part 3 (16661:) (" " is the last)


In [18]:
crunch_classified_p1 = pd.read_csv("crunch_classified_p1.csv", sep='\t', encoding='utf-8')
crunch_classified_p2 = pd.read_csv("crunch_classified_p2.csv", sep='\t', encoding='utf-8')
crunch_classified_p3 = pd.read_csv("crunch_classified_p3.csv", sep='\t', encoding='utf-8')

In [19]:
crunch_classified = crunch_classified_p1.append(crunch_classified_p2)
crunch_classified = crunch_classified.append(crunch_classified_p3)
crunch_classified

,Organization Name,Organization Name URL,Full Description,Industries,Website,Headquarters Location,Description,CB Rank (Company),SEMrush - Monthly Visits,SEMrush - Average Visits (6 months),...,Aberdeen - IT Spend,Aberdeen - IT Spend Currency,Aberdeen - IT Spend Currency (in USD),PP URL,DPO,Purpose,Acquired data,Data sharing,Rights,PP text
0,Exact,https://www.crunchbase.com/organization/exact,Exact is a global supplier of cloud business s...,"Accounting, Cloud Computing, CRM, Enterprise R...",http://www.exact.com,"Delft, Zuid-Holland, The Netherlands",Exact is a company that provides cloud based b...,"229,780","622,387","672,910.5",...,NaN,NaN,NaN,https://www.exact.com/privacy-statement,1,1,0,1,1,['exact hold BV a compani regist in the nether...
1,Xperience,https://www.crunchbase.com/organization/xperie...,Xperience provides software solutions within E...,"Cloud Computing, Consulting, CRM, Enterprise R...",https://www.xperience-group.com/,"Antrim, Antrim, United Kingdom",Xperience provides software solutions within E...,"187,002","402,949","198,967.83",...,6256711.0,USD,6256711.0,https://www.xperience-group.com/site-pages/pri...,1,1,1,1,1,['thi privaci statement is effect as of januar...
2,CRS4,https://www.crunchbase.com/organization/crs4,"Established in 1990, CRS4 (Center for Advanced...","Business Development, Cloud Computing, Educati...",http://www.crs4.it/,"Pula, Sardegna, Italy",CRS4 is an interdisciplinary research center p...,"764,573","103,411","58,347.33",...,NaN,NaN,NaN,https://www.crs4.it/privacy-policy/,1,0,0,0,1,['notic regard the process OF person data of u...
3,Siav,https://www.crunchbase.com/organization/siav,NaN,"Cloud Computing, Information Technology, Software",https://www.siav.com,"Rubano, Veneto, Italy","Siav provides digital storage, document manage...","524,480","40,997","17,044.83",...,3849039.0,USD,3849039.0,https://www.siav.com/privacy-policy/,1,1,0,0,1,"['', 'purpos of the process of person data art..."
4,Olivetti,https://www.crunchbase.com/organization/olivetti,Olivetti provides information technology produ...,"Cloud Computing, Cloud Data Services, Computer...",http://www.olivetti.com/,"Ivrea, Piemonte, Italy",Olivetti provides information technology produ...,"265,011","17,219","36,368.67",...,NaN,NaN,NaN,https://www.olivetti.com/en/web-privacy-policy...,1,0,0,0,1,['the purpos of thi privaci polici is to descr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1213,Roanoke Valley Broadband Authority,https://www.crunchbase.com/organization/roanok...,"RVBA offers local businesses secure, dependabl...","Cloud Storage, Internet, Optical Communication...",https://rvba.online,"Roanoke, Virginia, United States",Roanoke Valley Broadband Authority is a intern...,"1,901,822",NaN,NaN,...,NaN,NaN,NaN,https://rvba.online/privacy-policy/,1,1,1,1,1,['thi privaci notic for roanok valley broadban...
1214,Data Security Corp,https://www.crunchbase.com/organization/data-s...,NaN,"Cloud Storage, Computer, Data Storage, Network...",https://www.datasecuritycorp.com,"Tacoma, Washington, United States",Data Security Corp provides solutions to recor...,"1,909,261",NaN,NaN,...,NaN,NaN,NaN,https://www.datasecuritycorp.com/privacy.php,1,0,1,1,0,"['privaci IS AN import issu', 'pleas read thi ..."
1215,archivedi,https://www.crunchbase.com/organization/archivedi,NaN,"Cloud Storage, Software",https://archivedi.com,"Sarasota, Florida, United States",archivedi is a supplier of cloud-based enterpr...,"1,929,231",NaN,NaN,...,NaN,NaN,NaN,https://www.archivedi.com/en/terms,1,0,1,1,1,"['term of servic', 'By regist at archivedicom ..."
1216,BackupAddict,https://www.crunchbase.com/organization/backup...,NaN,"Cloud Storage, Information Technology, Profess...",https://www.backupaddict.com/,"Jersey Shore, Pennsylvania, United States",BackupAddict renders online backup plans to pr...,"2,145,360",NaN,NaN,...,NaN,NaN,NaN,https://www.backupaddict.com/privacy-policy.shtml,0,1,1,1,1,"['privaci polici effect da

In [20]:
if len(crunch_classified['Organization Name'].unique()) < len(crunch_classified.index):
    print("We have duplicates")

We have duplicates


In [21]:
len(crunch_classified['Organization Name'].unique())

4406

In [22]:
len(crunch_classified.drop_duplicates(subset='Organization Name', keep='first'))

4406

In [23]:
crunch_classified = crunch_classified.drop_duplicates(subset='Organization Name', keep='first').copy(deep=True)

In [24]:
crunch_classified

,Organization Name,Organization Name URL,Full Description,Industries,Website,Headquarters Location,Description,CB Rank (Company),SEMrush - Monthly Visits,SEMrush - Average Visits (6 months),...,Aberdeen - IT Spend,Aberdeen - IT Spend Currency,Aberdeen - IT Spend Currency (in USD),PP URL,DPO,Purpose,Acquired data,Data sharing,Rights,PP text
0,Exact,https://www.crunchbase.com/organization/exact,Exact is a global supplier of cloud business s...,"Accounting, Cloud Computing, CRM, Enterprise R...",http://www.exact.com,"Delft, Zuid-Holland, The Netherlands",Exact is a company that provides cloud based b...,"229,780","622,387","672,910.5",...,NaN,NaN,NaN,https://www.exact.com/privacy-statement,1,1,0,1,1,['exact hold BV a compani regist in the nether...
1,Xperience,https://www.crunchbase.com/organization/xperie...,Xperience provides software solutions within E...,"Cloud Computing, Consulting, CRM, Enterprise R...",https://www.xperience-group.com/,"Antrim, Antrim, United Kingdom",Xperience provides software solutions within E...,"187,002","402,949","198,967.83",...,6256711.0,USD,6256711.0,https://www.xperience-group.com/site-pages/pri...,1,1,1,1,1,['thi privaci statement is effect as of januar...
2,CRS4,https://www.crunchbase.com/organization/crs4,"Established in 1990, CRS4 (Center for Advanced...","Business Development, Cloud Computing, Educati...",http://www.crs4.it/,"Pula, Sardegna, Italy",CRS4 is an interdisciplinary research center p...,"764,573","103,411","58,347.33",...,NaN,NaN,NaN,https://www.crs4.it/privacy-policy/,1,0,0,0,1,['notic regard the process OF person data of u...
3,Siav,https://www.crunchbase.com/organization/siav,NaN,"Cloud Computing, Information Technology, Software",https://www.siav.com,"Rubano, Veneto, Italy","Siav provides digital storage, document manage...","524,480","40,997","17,044.83",...,3849039.0,USD,3849039.0,https://www.siav.com/privacy-policy/,1,1,0,0,1,"['', 'purpos of the process of person data art..."
4,Olivetti,https://www.crunchbase.com/organization/olivetti,Olivetti provides information technology produ...,"Cloud Computing, Cloud Data Services, Computer...",http://www.olivetti.com/,"Ivrea, Piemonte, Italy",Olivetti provides information technology produ...,"265,011","17,219","36,368.67",...,NaN,NaN,NaN,https://www.olivetti.com/en/web-privacy-policy...,1,0,0,0,1,['the purpos of thi privaci polici is to descr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1213,Roanoke Valley Broadband Authority,https://www.crunchbase.com/organization/roanok...,"RVBA offers local businesses secure, dependabl...","Cloud Storage, Internet, Optical Communication...",https://rvba.online,"Roanoke, Virginia, United States",Roanoke Valley Broadband Authority is a intern...,"1,901,822",NaN,NaN,...,NaN,NaN,NaN,https://rvba.online/privacy-policy/,1,1,1,1,1,['thi privaci notic for roanok valley broadban...
1214,Data Security Corp,https://www.crunchbase.com/organization/data-s...,NaN,"Cloud Storage, Computer, Data Storage, Network...",https://www.datasecuritycorp.com,"Tacoma, Washington, United States",Data Security Corp provides solutions to recor...,"1,909,261",NaN,NaN,...,NaN,NaN,NaN,https://www.datasecuritycorp.com/privacy.php,1,0,1,1,0,"['privaci IS AN import issu', 'pleas read thi ..."
1215,archivedi,https://www.crunchbase.com/organization/archivedi,NaN,"Cloud Storage, Software",https://archivedi.com,"Sarasota, Florida, United States",archivedi is a supplier of cloud-based enterpr...,"1,929,231",NaN,NaN,...,NaN,NaN,NaN,https://www.archivedi.com/en/terms,1,0,1,1,1,"['term of servic', 'By regist at archivedicom ..."
1216,BackupAddict,https://www.crunchbase.com/organization/backup...,NaN,"Cloud Storage, Information Technology, Profess...",https://www.backupaddict.com/,"Jersey Shore, Pennsylvania, United States",BackupAddict renders online backup plans to pr...,"2,145,360",NaN,NaN,...,NaN,NaN,NaN,https://www.backupaddict.com/privacy-policy.shtml,0,1,1,1,1,"['privaci polici effect da

# Manual labelling

## Traning set

In [62]:
training_indices = random.sample(range(0, len(crunch_classified)), 50)
training_indices

[3785,
 4225,
 3189,
 3253,
 431,
 314,
 879,
 2746,
 4118,
 4199,
 90,
 3322,
 632,
 1332,
 1686,
 2163,
 1000,
 1128,
 2024,
 1904,
 2610,
 1360,
 4221,
 848,
 3489,
 225,
 3704,
 281,
 626,
 10,
 260,
 2319,
 3853,
 1230,
 2372,
 306,
 1671,
 222,
 4001,
 2467,
 2906,
 992,
 403,
 1036,
 2220,
 110,
 847,
 1981,
 92,
 2877]

In [135]:
training_indices = [3785,
 4225,
 3189,
 3253,
 431,
 314,
 879,
 2746,
 4118,
 4199,
 90,
 3322,
 632,
 1332,
 1686,
 2163,
 1000,
 1128,
 2024,
 1904,
 2610,
 1360,
 4221,
 848,
 3489,
#  225,
 3704,
 281,
 626,
 10,
 260,
 2319,
 3853,
 1230,
 2372,
 306,
 1671,
 222,
#  4001,
 2467,
 2906,
 992,
 403,
 1036,
 2220,
 110,
 847,
 1981,
 92,
 2877]

In [142]:
training_set = crunch_classified.iloc[training_indices]

In [143]:
training_set

,Organization Name,Organization Name URL,Full Description,Industries,Website,Headquarters Location,Description,CB Rank (Company),SEMrush - Monthly Visits,SEMrush - Average Visits (6 months),...,Aberdeen - IT Spend,Aberdeen - IT Spend Currency,Aberdeen - IT Spend Currency (in USD),PP URL,DPO,Purpose,Acquired data,Data sharing,Rights,PP text
458,Flowrev,https://www.crunchbase.com/organization/flowrev,NaN,"Accounting, Cloud Infrastructure, Software",http://www.flowrev.com,"Portland, Oregon, United States",Flowrev is cost and revenue recognition and li...,"588,369",NaN,NaN,...,NaN,NaN,NaN,https://www.flowrev.com/privacy,0,1,1,1,1,"['privaci polici', 'thi privaci polici appli t..."
1007,Spearhead Technology,https://www.crunchbase.com/organization/spearh...,NaN,"Artificial Intelligence, Blockchain, Cloud Man...",https://spearheadtech.io,"Dallas, Texas, United States",Spearhead Technology is an IT firm that offers...,"595,349",NaN,NaN,...,NaN,NaN,NaN,https://spearheadtech.io/privacy-policy/,1,1,1,1,0,['the polici is design to assist you in unders...
2193,Data Everywhere,https://www.crunchbase.com/organization/data-e...,Data Everywhere is an add-in that enables spre...,"Cloud Data Services, Database, Publishing",https://www.dataeverywhere.com/,"Chicago, Illinois, United States",Data Everywhere enables users to manage and sh...,"150,947","9,113","6,441.83",...,NaN,NaN,NaN,https://www.dataeverywhere.com/privacy/,1,1,1,1,1,['thi privaci polici provid our polici and pro...
2257,Qumulex,https://www.crunchbase.com/organization/qumulex,"Qumulex is designed for security integrators, ...","Cloud Data Services, Information Technology, I...",https://www.qumulex.com,"Fishers, Indiana, United States",Qumulex is a Cloud and IoT-based video surveil...,"51,982","1,460",NaN,...,NaN,NaN,NaN,https://www.qumulex.com/privacy-policy,1,1,0,1,1,['qumulex inc qumulex us our or we is commit t...
432,Workspace 365,https://www.crunchbase.com/organization/worksp...,"Started out in 2010, Workspace 365 has always ...","Cloud Computing, Enterprise Software, Mobile, ...",https://workspace365.net/,"Nijkerk, Gelderland, The Netherlands",Workspace 365 is an internet based company.,"217,048","143,477","84,929.83",...,NaN,NaN,NaN,https://workspace365.net/en/privacy-terms-and-...,0,0,0,0,0,"['privaci polici for', 'latest updat octob', '..."
315,Cogmento,https://www.crunchbase.com/organization/cogmento,Cogmento CRM powered by Blockchain.,"Cloud Computing, CRM, Predictive Analytics, So...",https://cogmento.com,"San Francisco, California, United States",Cloud CRM,"942,680","1,032","3,661.5",...,NaN,NaN,NaN,https://cogmento.com/privacy.html,1,1,1,1,0,"['your data IS alway privat', 'what person ide..."
880,Skyen.,https://www.crunchbase.com/organization/skyen,NaN,"Cloud Computing, Cyber Security, IT Management",https://skyen.com/,"San Francisco, California, United States","Skyen deals with managed IT services, IT suppo...","1,025,833",NaN,NaN,...,NaN,NaN,NaN,https://skyen.com/privacy-policy/,1,1,1,1,0,"['skyen onlin privaci polici', 'privaci and da..."
1747,Kinamo,https://www.crunchbase.com/organization/kinamo,NaN,"Cloud Data Services, Network Hardware, Web Hos...",http://www.kinamo.be,"Antwerpen, Antwerpen, Belgium",Kinamo is an IT firm that provides services re...,"1,020,964","15,591","14,030.17",...,NaN,NaN,NaN,https://www.kinamo.be/en/privacy-statement,1,1,1,0,1,"['when will we save your data', '', 'when esta..."
873,AppShark Software,https://www.crunchbase.com/organization/appsha...,AppShark Software is a Salesforce consulting a...,"Cloud Management, Consulting, Information Tech...",https://www.appshark.com,"Dallas, Texas, United States",AppShark Software is a Salesforce consulting a...,"178,462","3,494","4,049.33",...,82172.0,USD,82172.0,https://www.appshark.com/privacy-policy/,1,1,1,1,1,"['', 'how DO WE handl your social login', 'In ..."
977,Virtual Service Operations,https://www.crunchbase.com/organization/virtua...,NaN,"Cloud Management, Consulting, Information Tech..."

In [112]:
crunch_classified.iloc[[4001]]

,Organization Name,Organization Name URL,Full Description,Industries,Website,Headquarters Location,Description,CB Rank (Company),SEMrush - Monthly Visits,SEMrush - Average Visits (6 months),...,Aberdeen - IT Spend,Aberdeen - IT Spend Currency,Aberdeen - IT Spend Currency (in USD),PP URL,DPO,Purpose,Acquired data,Data sharing,Rights,PP text
718,BetterGov,https://www.crunchbase.com/organization/bettergov,BetterGov helps the public sector in the digit...,"Cloud Management, Data Center Automation, Mana...",http://www.bettergov.co.uk,"London, England, United Kingdom",BetterGov is a consulting agency that speciali...,"1,677,700",NaN,NaN,...,NaN,NaN,NaN,https://www.bettergov.co.uk/legal_documents/pr...,1,0,1,0,1,"['privaci polici of wwwbettergovcouk', 'thi we..."


In [116]:
training_set_classification = [
1, #https://www.flowrev.com/privacy	
0, #https://spearheadtech.io/privacy-policy/
1, #https://www.dataeverywhere.com/privacy/	
1, #https://www.qumulex.com/privacy-policy	
1, #https://workspace365.net/en/privacy-terms-and-conditions-of-this-site/
1, #https://cogmento.com/privacy.html
0, #https://skyen.com/privacy-policy/
1, #https://www.kinamo.be/en/privacy-statement	
1, #https://www.appshark.com/privacy-policy/	
0, #https://www.vso-inc.com/privacy-policy/	
1, #https://www.openlinksw.com/privacy/	
0, #https://www.predikly.com/privacy-policy.html	
1, #https://www.fluentstream.com/privacy-policy/
0, #https://kasmo.co/privacy-policy/
1, #https://tlmpartners.com/privacy-policy/
1, #https://www.eenetworks.com/privacy-policy/
1, #https://littlevista.ie/privacy-notice/	
0, #https://www.cloudamize.com/en/legal/privacy-notice/
0, #https://customercaregroup.co.uk/privacy-policy/	
1, #https://www.fdtgroup.org/privacy-policy/
1, #https://realto.ai/privacy-policy/
1, #https://nrby.com/privacy-policy/	
0, #https://brewcitypc.com/privacy-policy/
0, #https://www.sintroncorp.com/privacy-policy.htm
0, #https://easytech.com.cy/privacy-policy	
# 0, # ERUIT https://www.encora.com/insights/privacy-preser
0, #https://www.violinsystems.com/privacy-policy/	
1, #https://www.netfactor.com/legal/privacy-policy/	
0, #https://silverlinecrm.com/privacy-policy/	
1, #https://www.transputec.com/privacy-policy/
1, #https://daminion.net/company/privacy-policy/
0, #https://prisomtechnology.com/privacy-policy/
0, #https://www.chalkline.tech/privacy-policy
1, #https://corezoid.com/terms-and-conditions/
1, #https://newcytech.logicom.net/privacy-policy/
0, #https://www.condreycorp.com/privacy_policy
1, #https://loft.sh/legal/privacy-policy/
1, #https://www.sugarcrm.com/legal/privacy-policy/
# 0, # ERUIT https://www.bettergov.co.uk/legal_documents/pr...	
0, #https://www.motivit.com/privacy-policy/
1, #https://www.gocanvas.com/content/about-us/policy/canvas-terms-of-service/
0, #https://interworks.cloud/privacy-policy/
0, #https://endecom.com/privacy-policy-terms-of-use/
1, #https://64teq.com/privacy-policy/
1, #https://www.onmo.com/privacy-policy/	
0, #https://www.pavelcomm.com/privacy-policy/
0, #https://www.exnoweb.com/privacypolicy.php
0, #https://xenogenix.co.uk/privacy-policy-cookies/	
0, #https://www.mattnj.com/privacy-policy
1, #https://prezi.com/legal/privacy-policy/	
]

In [134]:
training_set_classification

[1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1]

In [30]:
# crunch_labeled_s1 = pd.read_excel("validation/validation.xlsx", sheet_name="Sample 1")
# crunch_labeled_s2 = pd.read_excel("validation/validation.xlsx", sheet_name="Sample 2")

In [37]:
# cname_list_s1 = crunch_labeled_s1['Organization Name'].tolist()
# cname_list_s2 = crunch_labeled_s2['Organization Name'].tolist()
# cname_list
# crunch_classified[crunch_classified['Organization Name'].isin(cname_list_s1)]

['Concise Computer Consulting',
 'Bmore Technology',
 'Elevated Computing',
 'Fuze, an 8x8 company',
 'Systum',
 'OnlyLink',
 'Integrated Reasoning',
 'VivSoft Technologies',
 'Gain Innovation',
 'CloudMasters',
 'Xano',
 'Mercatus',
 'Kabuto',
 'Maus Software',
 'Oy Silvadata Ab',
 'Quicket Solutions',
 'Axcelinno',
 'Fortunesoft IT Innovations,Inc',
 'The Shattuck Group',
 'HireHop',
 'Easy HR',
 'Nebulait',
 'Trainbit',
 'IT&C EXPERTS NETWORK SRL',
 'Level 12',
 'TCC The Computer Company',
 'Zymbian',
 'Soft Skills',
 'SonicBee',
 'BetaBulls',
 'Inovitech',
 'AskDelphi',
 'Constellation',
 'DeNet',
 'EdEazy',
 'Infodidac',
 'RedBrick IT',
 'jPALIO',
 'Voipwerk',
 'Data Cube Systems',
 'toplink',
 'Intelligent Automation',
 'Alta Technologies',
 'Intelisa',
 'SI-NET',
 'Link Solutions',
 'OP-Rate',
 'Yodiz',
 'HeyWire',
 'Storage Made Easy']

### Test set

In [138]:
# training_indices

test_indices = []

while len(test_indices) < 50: # Where 6 can be replaced with how many numbers you want minus 1
    x = random.sample(range(0, len(crunch_classified)), 1).pop()
    if x not in training_indices:
        test_indices.append(x)

In [182]:
test_indices

[1708,
 1347,
 4354,
 3054,
 1271,
 1659,
 4174,
 3602,
 2812,
 2786,
 2016,
 2152,
 3401,
 2188,
 782,
 2309,
 4382,
 1192,
 1707,
 3813,
 3028,
 1797,
 3029,
 3997,
 3085,
 3172,
 2036,
 133,
 3420,
 511,
 930,
 1111,
 1580,
 932,
 4375,
 1396,
 3643,
 3336,
 3703,
 3086,
 384,
 1450,
 3403,
 1274,
 1434,
 3575,
 861,
 168,
 2023,
 1276]

In [183]:
test_indices = [1708,
#  1347,
 4354,
 3054,
 1271,
 1659,
 4174,
 3602,
 2812,
 2786,
 2016,
#  2152,
 3401,
 2188,
 782,
 2309,
 4382,
 1192,
 1707,
 3813,
 3028,
 1797,
 3029,
 3997,
 3085,
 3172,
 2036,
 133,
 3420,
 511,
 930,
 1111,
 1580,
 932,
 4375,
 1396,
 3643,
 3336,
 3703,
 3086,
 384,
 1450,
 3403,
 1274,
 1434,
 3575,
 861,
 168,
 2023,
 1276]

In [184]:
test_set = crunch_classified.iloc[test_indices]
test_set

,Organization Name,Organization Name URL,Full Description,Industries,Website,Headquarters Location,Description,CB Rank (Company),SEMrush - Monthly Visits,SEMrush - Average Visits (6 months),...,Aberdeen - IT Spend,Aberdeen - IT Spend Currency,Aberdeen - IT Spend Currency (in USD),PP URL,DPO,Purpose,Acquired data,Data sharing,Rights,PP text
1710,GSD Venture Studios,https://www.crunchbase.com/organization/gsd-ve...,For too long self-motivated entrepreneurs have...,"Cloud Computing, Cyber Security, EdTech, FinTe...",https://www.gsdvs.com,"Menlo Park, California, United States",Venture studio specializing in helping resilie...,"93,853","6,910","4,599.67",...,NaN,NaN,NaN,https://www.gsdvs.com/privacy-policy,1,0,1,1,0,"['privaci polici', '\u200b', 'thi privaci poli..."
1158,Dokkio,https://www.crunchbase.com/organization/dokkio,"Dokkio is an intelligent, cloud-based file ass...","Artificial Intelligence, Cloud Storage, Intern...",https://dokkio.com/,"San Mateo, California, United States","Find, organize, and understand all your cloud ...","61,996","11,979","11,949.5",...,NaN,NaN,NaN,https://www.dokkio.com/privacy,1,1,1,1,1,"['privaci polici', 'dokkio inc', 'dokkio we or..."
2057,Silver Linings Technology,https://www.crunchbase.com/organization/silver...,NaN,"Business Intelligence, Cloud Data Services, Cl...",https://www.silverliningstechnology.com,"Thousand Oaks, California, United States","Silver Linings Technologies offers managed IT,...","745,959",NaN,NaN,...,NaN,NaN,NaN,https://www.silverliningstechnology.com/2017/0...,0,0,0,0,0,['for year the concept of bring your own devic...
1272,Ordr,https://www.crunchbase.com/organization/ordr-aec7,Ordr develops an AI-based systems control engi...,"Artificial Intelligence, Cloud Computing, Cybe...",https://ordr.net,"Santa Clara, California, United States",Ordr is a connected device cybersecurity start...,"4,733","13,582","10,768.83",...,179458.0,USD,179458.0,https://ordr.net/privacy-policy/,1,0,1,1,1,"['the ordr privaci polici wa updat on april', ..."
1661,Solo.io,https://www.crunchbase.com/organization/solo-io,Solo.io develops tools to help the enterprise ...,"Apps, Cloud Computing, Developer APIs, Interne...",https://www.solo.io,"Cambridge, Massachusetts, United States",Solo.io develops tools to help the enterprise ...,"1,898","179,075","118,448.67",...,NaN,NaN,NaN,https://www.solo.io/privacy-policy/,1,1,1,1,1,"['privaci polici', 'introduct', 'thi privaci p..."
943,Dasinfomedia,https://www.crunchbase.com/organization/dasinf...,Dasinfomedia is a team of experienced develope...,"Apps, Cloud Management, Enterprise Resource Pl...",https://www.dasinfomedia.com/,"Maryland, Louisiana, United States",Das infomedia INC offers innovative IT solutio...,"317,112",288,"1,647.83",...,NaN,NaN,NaN,https://www.dasinfomedia.com/privacy-policy/,0,1,1,0,1,['We are commit to maintain the accuraci confi...
190,Ambassador Labs,https://www.crunchbase.com/organization/datawire,Ambassador Labs builds a continuous deployment...,"Cloud Infrastructure, Computer, IT Infrastruct...",https://www.getambassador.io/,"Boston, Massachusetts, United States",Ambassador Labs is making it easy for develope...,"2,578","183,775","103,453.83",...,NaN,NaN,NaN,https://www.getambassador.io/privacy-policy/,0,0,0,0,0,"['privaci polici', 'datawir', 'term of servic ..."
1814,CLICK ON Hungary,https://www.crunchbase.com/organization/click-...,NaN,"Cloud Data Services, Cloud Management, Softwar...",https://click-on.hu,"Budapest, Budapest, Hungary",CLICK ON Hungary is a software development com...,"1,635,213",124,NaN,...,NaN,NaN,NaN,https://click-on.hu/cookie-policy-eu/,1,0,1,0,1,"['thi cooki polici wa last updat on', 'and app..."
1788,Chinsay AB,https://www.crunchbase.com/organization/chinsa...,"Chinsay is the leading, independent cloud plat...","Cloud Data Services, Internet, SaaS, Software",https://www.chinsay.com/,"Stockholm, Stockholms Lan, Sweden","Chinsay is the leading, independent cloud plat...","61,800",880,"189,605.83",...,13583947.

In [185]:
test_set_classification = [
1, #https://www.gsdvs.com/privacy-policy	
# 0, #WERKT NIET; ERUIT
1, #https://www.dokkio.com/privacy
0, #https://www.silverliningstechnology.com/2017/0...	
1, #https://ordr.net/privacy-policy/
1, #https://www.solo.io/privacy-policy/	
0, #https://www.dasinfomedia.com/privacy-policy/	
1, #https://www.getambassador.io/privacy-policy/	
0, #https://click-on.hu/cookie-policy-eu/	
0, #https://www.chinsay.com/privacy-policy	
0, #https://www.neos-it.com/privacy-policy/	
# 0, #ERUIT https://www.christoit.com/managed-it-services-...	
1, #https://onefede.com/privacy-policy/	
0, #https://www.geminicomputersystems.com/privacy-and-communications-policy/
1, #https://www.celigo.com/privacy/	
0, #https://www.stspl.com/company/privacy-policy/	
1, #https://www.netmass.com/privacy-policy/	
0, #https://jbtechent.com/privacy-policy/
1, #https://claritask.com/privacy/	
1, #https://www.sleepyengineers.com/policies/tos.html
1, #https://promo.com/privacy-policy?utm_content=for_slidely-users_footer
1, #https://www.carebandremembers.com/privacy-policy/	
1, #https://www.irevomm.com/privacy.html	
0, #https://parallel-it.com/privacy/	
0, #https://www.onepointsync.com/privacy-policy/	
1, #https://www.traceable.ai/privacy-policy	
1, #https://www.k2.com/legal/privacy-policy?lang=en	
1, #https://palmbeachsoftware.com/privacy-policy/	
0, #https://www.ideationts.com/index.php/privacy-p...	
0, #https://www.accessit.gr/en/privacy-policy-en	
1, #https://dataprius.com/en/dataprius-legal-advic...	
1, #https://www.koding.com/legal/copyright
1, #https://radian.systems/privacy	
1, #https://www.veset.tv/privacy-policy	
0, #https://scalenorth.com/privacy-statement-us/	
0, #https://zuggand.com/privacy-policy/	
0, #https://www.trianz.com/privacy-statement	
0, #https://www.datamapllc.com/privacy	
1, #https://www.monarchconnected.com/privacy-policy/	
0, #https://www.c3corp.us/legal/privacy-policy/	
0, #https://www.metagyre.com/privacy-policy/	
1, #https://www.simplerqms.com/privacy-policy/	
0, #https://www.webhostingbingo.com/privacy-policy	
1, #https://cloudmylab.com/privacy-policies/	
0, #https://princh.com/privacy-policy/#.Y4SdJHbMKUk
0, #https://www.autointelli.com/privacy-policy.html	
1, #https://zenware.com/policy-harvest-bin/	
0, #https://microxpress.net/privacy-policy/	
0, #https://technologycentres.co.uk/privacy/
1, #https://www.pulsehealth.tech/privacy-policy.html	
]

# Identify bi- unigrams

In [31]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

In [145]:
training_set["relevant"] = training_set_classification

<ipython-input-145-0ef470eb2413>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_set["relevant"] = training_set_classification


In [146]:
training_set

,Organization Name,Organization Name URL,Full Description,Industries,Website,Headquarters Location,Description,CB Rank (Company),SEMrush - Monthly Visits,SEMrush - Average Visits (6 months),...,Aberdeen - IT Spend Currency,Aberdeen - IT Spend Currency (in USD),PP URL,DPO,Purpose,Acquired data,Data sharing,Rights,PP text,relevant
458,Flowrev,https://www.crunchbase.com/organization/flowrev,NaN,"Accounting, Cloud Infrastructure, Software",http://www.flowrev.com,"Portland, Oregon, United States",Flowrev is cost and revenue recognition and li...,"588,369",NaN,NaN,...,NaN,NaN,https://www.flowrev.com/privacy,0,1,1,1,1,"['privaci polici', 'thi privaci polici appli t...",1
1007,Spearhead Technology,https://www.crunchbase.com/organization/spearh...,NaN,"Artificial Intelligence, Blockchain, Cloud Man...",https://spearheadtech.io,"Dallas, Texas, United States",Spearhead Technology is an IT firm that offers...,"595,349",NaN,NaN,...,NaN,NaN,https://spearheadtech.io/privacy-policy/,1,1,1,1,0,['the polici is design to assist you in unders...,0
2193,Data Everywhere,https://www.crunchbase.com/organization/data-e...,Data Everywhere is an add-in that enables spre...,"Cloud Data Services, Database, Publishing",https://www.dataeverywhere.com/,"Chicago, Illinois, United States",Data Everywhere enables users to manage and sh...,"150,947","9,113","6,441.83",...,NaN,NaN,https://www.dataeverywhere.com/privacy/,1,1,1,1,1,['thi privaci polici provid our polici and pro...,1
2257,Qumulex,https://www.crunchbase.com/organization/qumulex,"Qumulex is designed for security integrators, ...","Cloud Data Services, Information Technology, I...",https://www.qumulex.com,"Fishers, Indiana, United States",Qumulex is a Cloud and IoT-based video surveil...,"51,982","1,460",NaN,...,NaN,NaN,https://www.qumulex.com/privacy-policy,1,1,0,1,1,['qumulex inc qumulex us our or we is commit t...,1
432,Workspace 365,https://www.crunchbase.com/organization/worksp...,"Started out in 2010, Workspace 365 has always ...","Cloud Computing, Enterprise Software, Mobile, ...",https://workspace365.net/,"Nijkerk, Gelderland, The Netherlands",Workspace 365 is an internet based company.,"217,048","143,477","84,929.83",...,NaN,NaN,https://workspace365.net/en/privacy-terms-and-...,0,0,0,0,0,"['privaci polici for', 'latest updat octob', '...",1
315,Cogmento,https://www.crunchbase.com/organization/cogmento,Cogmento CRM powered by Blockchain.,"Cloud Computing, CRM, Predictive Analytics, So...",https://cogmento.com,"San Francisco, California, United States",Cloud CRM,"942,680","1,032","3,661.5",...,NaN,NaN,https://cogmento.com/privacy.html,1,1,1,1,0,"['your data IS alway privat', 'what person ide...",1
880,Skyen.,https://www.crunchbase.com/organization/skyen,NaN,"Cloud Computing, Cyber Security, IT Management",https://skyen.com/,"San Francisco, California, United States","Skyen deals with managed IT services, IT suppo...","1,025,833",NaN,NaN,...,NaN,NaN,https://skyen.com/privacy-policy/,1,1,1,1,0,"['skyen onlin privaci polici', 'privaci and da...",0
1747,Kinamo,https://www.crunchbase.com/organization/kinamo,NaN,"Cloud Data Services, Network Hardware, Web Hos...",http://www.kinamo.be,"Antwerpen, Antwerpen, Belgium",Kinamo is an IT firm that provides services re...,"1,020,964","15,591","14,030.17",...,NaN,NaN,https://www.kinamo.be/en/privacy-statement,1,1,1,0,1,"['when will we save your data', '', 'when esta...",1
873,AppShark Software,https://www.crunchbase.com/organization/appsha...,AppShark Software is a Salesforce consulting a...,"Cloud Management, Consulting, Information Tech...",https://www.appshark.com,"Dallas, Texas, United States",AppShark Software is a Salesforce consulting a...,"178,462","3,494","4,049.33",...,USD,82172.0,https://www.appshark.com/privacy-policy/,1,1,1,1,1,"['', 'how DO WE handl your social login', 'In ...",1
977,Virtual Service Operations,https://www.crunchbase.com/organization/virtua...,NaN,"Cloud Management, Consulting, Information Tech...",https://vso-inc.com,"Nokesville,

In [148]:
from sklearn.feature_extraction.text import CountVectorizer
c_vec = CountVectorizer(stop_words=stopwords.words('english'), ngram_range=(2,3))
# matrix of ngrams
ngrams = c_vec.fit_transform(training_set['PP text'])
# count frequency of ngrams
count_values = ngrams.toarray().sum(axis=0)
# list of ngrams
vocab = c_vec.vocabulary_
df_ngram = pd.DataFrame(sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True)
            ).rename(columns={0: 'frequency', 1:'bigram/trigram'})

In [150]:
df_ngram.head(20)

,frequency,bigram/trigram
0,694,person data
1,630,person inform
2,415,third parti
3,399,privaci polici
4,278,thi privaci
5,207,thi privaci polici
6,173,inform collect
7,163,contact us
8,146,use servic
9,144,process person


In [151]:
c_vec = CountVectorizer(stop_words=stopwords.words('english'), ngram_range=(1,2))
tfidf_x

In [203]:
x_train = training_set['PP text']
y_train = training_set_classification

x_test = test_set['PP text']
y_test = test_set_classification

In [195]:
y_test.count(0)

23

In [197]:
x_train

458     ['privaci polici', 'thi privaci polici appli t...
1007    ['the polici is design to assist you in unders...
2193    ['thi privaci polici provid our polici and pro...
2257    ['qumulex inc qumulex us our or we is commit t...
432     ['privaci polici for', 'latest updat octob', '...
315     ['your data IS alway privat', 'what person ide...
880     ['skyen onlin privaci polici', 'privaci and da...
1747    ['when will we save your data', '', 'when esta...
873     ['', 'how DO WE handl your social login', 'In ...
977     ['privaci notic', 'last updat novemb', 'thank ...
90      ['openlink softwar privaci polici', 'As a pass...
2327    ['inform collect and use predikli may request ...
633     ['updat may', 'thank for your interest in flue...
1333    ['your privaci IS import TO kasmo technolog', ...
1688    ['privaci polici', 'last updat april', 'thi pr...
1156    ['We are eenetwork an IT consult compani', 'We...
1001    ['when you visit our websit or use our servic ...
1129    ['upda

In [220]:
from sklearn.model_selection import GridSearchCV
from pprint import pprint
from imblearn.over_sampling import RandomOverSampler
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline, make_pipeline
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import roc_auc_score, precision_score, accuracy_score, recall_score, average_precision_score, classification_report, confusion_matrix, f1_score, roc_auc_score, roc_curve, precision_recall_curve, plot_precision_recall_curve
from sklearn.model_selection import train_test_split

### LR

In [232]:
df_labeled = training_set.append(test_set)

In [234]:
training_set

,Organization Name,Organization Name URL,Full Description,Industries,Website,Headquarters Location,Description,CB Rank (Company),SEMrush - Monthly Visits,SEMrush - Average Visits (6 months),...,Aberdeen - IT Spend Currency,Aberdeen - IT Spend Currency (in USD),PP URL,DPO,Purpose,Acquired data,Data sharing,Rights,PP text,relevant
458,Flowrev,https://www.crunchbase.com/organization/flowrev,NaN,"Accounting, Cloud Infrastructure, Software",http://www.flowrev.com,"Portland, Oregon, United States",Flowrev is cost and revenue recognition and li...,"588,369",NaN,NaN,...,NaN,NaN,https://www.flowrev.com/privacy,0,1,1,1,1,"['privaci polici', 'thi privaci polici appli t...",1
1007,Spearhead Technology,https://www.crunchbase.com/organization/spearh...,NaN,"Artificial Intelligence, Blockchain, Cloud Man...",https://spearheadtech.io,"Dallas, Texas, United States",Spearhead Technology is an IT firm that offers...,"595,349",NaN,NaN,...,NaN,NaN,https://spearheadtech.io/privacy-policy/,1,1,1,1,0,['the polici is design to assist you in unders...,0
2193,Data Everywhere,https://www.crunchbase.com/organization/data-e...,Data Everywhere is an add-in that enables spre...,"Cloud Data Services, Database, Publishing",https://www.dataeverywhere.com/,"Chicago, Illinois, United States",Data Everywhere enables users to manage and sh...,"150,947","9,113","6,441.83",...,NaN,NaN,https://www.dataeverywhere.com/privacy/,1,1,1,1,1,['thi privaci polici provid our polici and pro...,1
2257,Qumulex,https://www.crunchbase.com/organization/qumulex,"Qumulex is designed for security integrators, ...","Cloud Data Services, Information Technology, I...",https://www.qumulex.com,"Fishers, Indiana, United States",Qumulex is a Cloud and IoT-based video surveil...,"51,982","1,460",NaN,...,NaN,NaN,https://www.qumulex.com/privacy-policy,1,1,0,1,1,['qumulex inc qumulex us our or we is commit t...,1
432,Workspace 365,https://www.crunchbase.com/organization/worksp...,"Started out in 2010, Workspace 365 has always ...","Cloud Computing, Enterprise Software, Mobile, ...",https://workspace365.net/,"Nijkerk, Gelderland, The Netherlands",Workspace 365 is an internet based company.,"217,048","143,477","84,929.83",...,NaN,NaN,https://workspace365.net/en/privacy-terms-and-...,0,0,0,0,0,"['privaci polici for', 'latest updat octob', '...",1
315,Cogmento,https://www.crunchbase.com/organization/cogmento,Cogmento CRM powered by Blockchain.,"Cloud Computing, CRM, Predictive Analytics, So...",https://cogmento.com,"San Francisco, California, United States",Cloud CRM,"942,680","1,032","3,661.5",...,NaN,NaN,https://cogmento.com/privacy.html,1,1,1,1,0,"['your data IS alway privat', 'what person ide...",1
880,Skyen.,https://www.crunchbase.com/organization/skyen,NaN,"Cloud Computing, Cyber Security, IT Management",https://skyen.com/,"San Francisco, California, United States","Skyen deals with managed IT services, IT suppo...","1,025,833",NaN,NaN,...,NaN,NaN,https://skyen.com/privacy-policy/,1,1,1,1,0,"['skyen onlin privaci polici', 'privaci and da...",0
1747,Kinamo,https://www.crunchbase.com/organization/kinamo,NaN,"Cloud Data Services, Network Hardware, Web Hos...",http://www.kinamo.be,"Antwerpen, Antwerpen, Belgium",Kinamo is an IT firm that provides services re...,"1,020,964","15,591","14,030.17",...,NaN,NaN,https://www.kinamo.be/en/privacy-statement,1,1,1,0,1,"['when will we save your data', '', 'when esta...",1
873,AppShark Software,https://www.crunchbase.com/organization/appsha...,AppShark Software is a Salesforce consulting a...,"Cloud Management, Consulting, Information Tech...",https://www.appshark.com,"Dallas, Texas, United States",AppShark Software is a Salesforce consulting a...,"178,462","3,494","4,049.33",...,USD,82172.0,https://www.appshark.com/privacy-policy/,1,1,1,1,1,"['', 'how DO WE handl your social login', 'In ...",1
977,Virtual Service Operations,https://www.crunchbase.com/organization/virtua...,NaN,"Cloud Management, Consulting, Information Tech...",https://vso-inc.com,"Nokesville,

In [235]:
y_labeled = training_set_classification + test_set_classification

In [236]:
x_train, x_test, y_train, y_test = train_test_split(df_labeled['PP text'], y_labeled, test_size=.2, 
                                                    random_state=42, stratify=y_labeled)

In [252]:
x_train, x_test, y_train, y_test = train_test_split(df_labeled['PP text'], y_labeled, test_size=.2, 
                                                    random_state=42, stratify=y_labeled)

vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'), ngram_range = (1,3))
x_train = vectorizer.fit_transform(x_train)
lr = LogisticRegression(max_iter = 100)
lr.fit(x_train, y_train)

x_test = vectorizer.transform(x_test)
y_pred = lr.predict(x_test)
y_prob = lr.predict_proba(x_test)[:,1]

print()
print("Most important features:")
words = vectorizer.get_feature_names()
for label, coefs, intercept in zip(lr.classes_, lr.coef_, lr.intercept_):
    print(label)
    sort_zipped_list = sorted(zip(words, coefs), key = lambda x: x[1], reverse = True) 
    for t, c in list(sort_zipped_list)[:10]:
        print(t, c)
    print("...")
    print("INTERCEPT:" +str(intercept))
    print("...")
    for t, c in list(sort_zipped_list)[-10:]:
        print(t, c)
    print()
    print()

print("Confusion matrix ")
print(confusion_matrix(y_test, y_pred))
print()
print("Classification report ")
print(classification_report(y_test, y_pred))


Most important features:
0
servic 0.3983552936150055
ordr 0.327512201435929
googl 0.32344733671052567
user 0.29179964368746386
nintex 0.265953785039558
sugarcrm 0.25655045741219734
soloio 0.2471223889868892
flowrev 0.21057586522185023
qumulex 0.20558902993562567
datawir 0.20283748371020469
...
INTERCEPT:0.08019957246080696
...
thi websit -0.15992570164882813
thi polici -0.17704628355559374
websit -0.19338443419845397
datamap -0.19986541319849369
predikli -0.2010600529487775
function name -0.21180369084135892
skyen -0.22783951615470385
person inform -0.278781149497863
endecom -0.2939447068280583
inform -0.3308318009033248


Confusion matrix 
[[2 7]
 [2 9]]

Classification report 
              precision    recall  f1-score   support

           0       0.50      0.22      0.31         9
           1       0.56      0.82      0.67        11

    accuracy                           0.55        20
   macro avg       0.53      0.52      0.49        20
weighted avg       0.53      0.55      

C:\Users\aaberkan\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


### Multinomial NB

In [213]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

In [265]:
# x_train = training_set['PP text']
# y_train = training_set_classification

# x_test = test_set['PP text']
# y_test = test_set_classification

x_train, x_test, y_train, y_test = train_test_split(df_labeled['PP text'], y_labeled, test_size=.3, 
                                                    random_state=48, stratify=y_labeled)

vect = CountVectorizer().fit(x_train)
X_train_vectorized = vect.transform(x_train)
clfrNB = MultinomialNB(alpha = 0.4)
clfrNB.fit(X_train_vectorized, y_train)
preds = clfrNB.predict(vect.transform(x_test))
score = roc_auc_score(y_test, preds)

print("AUC :" + str(score))

print("Confusion matrix ")
print(confusion_matrix(y_test, preds))
print()
print("Classification report ")
print(classification_report(y_test, preds))

AUC :0.719047619047619
Confusion matrix 
[[ 8  6]
 [ 2 13]]

Classification report 
              precision    recall  f1-score   support

           0       0.80      0.57      0.67        14
           1       0.68      0.87      0.76        15

    accuracy                           0.72        29
   macro avg       0.74      0.72      0.72        29
weighted avg       0.74      0.72      0.72        29

